In [48]:
!pip freeze > requirements.txt
# ! pip install -r requirements.txt # Uncomment this line to install dependencies in a new environment

In [37]:
# !pip install kagglehub

import kagglehub
import pandas as pd
from IPython.display import display

# # Download selected version
# path = kagglehub.dataset_download("shivamb/company-acquisitions-7-top-companies/versions/1")

# print("Path to dataset files:", path)

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/technology-mergers-and-acquisitions")

print("Path to dataset files:", path)

Path to dataset files: /Users/isaiah/.cache/kagglehub/datasets/joebeachcapital/technology-mergers-and-acquisitions/versions/1


In [39]:
# Load all CSVs
acquiring_companies = pd.read_csv("data/Acquiring Tech Companies.csv")
acquired_companies = pd.read_csv("data/Acquired Tech Companies.csv")
acquisitions = pd.read_csv("data/Acquisitions.csv")
founders_board = pd.read_csv("data/Founders and Board Members.csv")

# Link acquisitions table with acquiring companies
merged_1 = acquisitions.merge(
    acquiring_companies,
    how="left",
    left_on="Acquiring Company",
    right_on="CrunchBase Profile",
    suffixes=("", "_Acquirer")
)

# Link with acquired companies
merged_2 = merged_1.merge(
    acquired_companies,
    how="left",
    on="Acquisitions ID",
    suffixes=("", "_Acquired")
)

# You can also join founders/board members with acquiring companies
merged_full = merged_2.merge(
    founders_board,
    how="left",
    left_on="Acquiring Company",
    right_on="Companies"
)

# Now merged_full contains all the linked info
print(merged_full.head())


                                     Acquisitions ID     Acquired Company  \
0                      [24]7 acquired Tellme in 2012               Tellme   
1                         3Com acquired Palm in 1997                 Palm   
2  Adobe Systems acquired Accelio Corporation in ...  Accelio Corporation   
3  Adobe Systems acquired Accelio Corporation in ...  Accelio Corporation   
4  Adobe Systems acquired Accelio Corporation in ...  Accelio Corporation   

  Acquiring Company  Year of acquisition announcement Deal announced on  \
0             [24]7                              2012         1/02/2012   
1              3Com                              1997         1/06/1997   
2             Adobe                              2002        15/04/2002   
3             Adobe                              2002        15/04/2002   
4             Adobe                              2002        15/04/2002   

                Price       Status        Terms  \
0  Undisclosed amount  Undisclosed 

## Phase 1 (Censored Regression)

In [20]:
# keep only relevant columns
phase1_cols = [
    # dependent variable
    'Price',
    
    # target firm features
    'Year Founded_Acquired',
    'Market Categories_Acquired',
    'Number of Employees',
    'Total Funding ($)',
    'IPO',
    'Country (HQ)_Acquired',
    'State / Region (HQ)_Acquired',
    
    # acquirer features
    'Acquiring Company',
    'Number of Acquisitions',
    'Country (HQ)',
    
    # contextual / timing
    'Year of acquisition announcement',
    'Deal announced on',
    'Status',
    'Terms'
]

merged_phase1 = merged_full[phase1_cols].copy()

# preview the cleaned frame
merged_phase1.head()

,Price,Year Founded_Acquired,Market Categories_Acquired,Number of Employees,Total Funding ($),IPO,Country (HQ)_Acquired,State / Region (HQ)_Acquired,Acquiring Company,Number of Acquisitions,Country (HQ),Year of acquisition announcement,Deal announced on,Status,Terms
0,Undisclosed amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[24]7,NaN,NaN,2012,1/02/2012,Undisclosed,Undisclosed
1,Undisclosed amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3Com,NaN,NaN,1997,1/06/1997,Undisclosed,Undisclosed
2,Undisclosed amount,NaN,NaN,NaN,NaN,NaN,Canada,Ontario,Adobe,NaN,NaN,2002,15/04/2002,Undisclosed,Undisclosed
3,Undisclosed amount,NaN,NaN,NaN,NaN,NaN,Canada,Ontario,Adobe,NaN,NaN,2002,15/04/2002,Undisclosed,Undisclosed
4,Undisclosed amount,NaN,NaN,NaN,NaN,NaN,Canada,Ontario,Adobe,NaN,NaN,2002,15/04/2002,Undisclosed,Undisclosed


In [21]:
# rename columns for simplicity
merged_phase1.rename(columns={
    'Price': 'deal_value',
    'Year Founded_Acquired': 'year_founded_target',
    'Market Categories_Acquired': 'sector_target',
    'Number of Employees': 'employees_target',
    'Total Funding ($)': 'funding_target',
    'IPO': 'ipo_status',
    'Country (HQ)_Acquired': 'country_target',
    'Country (HQ)': 'country_acquirer',
    'Number of Acquisitions': 'acquirer_experience'
}, inplace=True)

# drop duplicates and rows with no key info
merged_phase1.drop_duplicates(inplace=True)
merged_phase1.dropna(subset=['deal_value', 'year_founded_target', 'sector_target'], how='all', inplace=True)


In [22]:
# remove currency symbols, commas, and text like 'USD' or 'M'
merged_phase1['deal_value'] = (
    merged_phase1['deal_value']
    .astype(str)
    .str.replace(r'[^0-9.\-]', '', regex=True)  # keep only digits and dots
)

# convert to float; invalid entries become NaN
merged_phase1['deal_value'] = pd.to_numeric(merged_phase1['deal_value'], errors='coerce')

# now re-run the censoring logic
merged_phase1['is_censored'] = merged_phase1['deal_value'].isna() | \
                               merged_phase1['deal_value'].astype(str).str.contains('undisclosed', case=False, na=False)

# compute threshold safely, ignoring NaN
D = merged_phase1['deal_value'].dropna().quantile(0.75)
merged_phase1['lower_bound'] = D


## One-Hot Encoding for Categorical Variables

In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# --- define categorical and numeric columns ---
cat_cols = ['sector_target', 'country_target', 'country_acquirer', 'Status', 'Terms']
num_cols = ['year_founded_target', 'employees_target', 'funding_target', 'acquirer_experience']

# --- verify columns exist before encoding ---
existing_cat_cols = [c for c in cat_cols if c in merged_phase1.columns]
existing_num_cols = [c for c in num_cols if c in merged_phase1.columns]

print("Categorical columns used:", existing_cat_cols)
print("Numeric columns used:", existing_num_cols)

# --- one-hot encode categorical ---
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_cats = pd.DataFrame(
    encoder.fit_transform(merged_phase1[existing_cat_cols]),
    columns=encoder.get_feature_names_out(existing_cat_cols),
    index=merged_phase1.index
)

# --- scale numeric ---
scaler = StandardScaler()
scaled_nums = pd.DataFrame(
    scaler.fit_transform(merged_phase1[existing_num_cols]),
    columns=existing_num_cols,
    index=merged_phase1.index
)

# --- combine encoded + scaled features ---
X = pd.concat([encoded_cats, scaled_nums], axis=1)

# --- define target and censoring indicator ---
y = merged_phase1['deal_value']
is_censored = merged_phase1['is_censored']


Categorical columns used: ['sector_target', 'country_target', 'country_acquirer', 'Status', 'Terms']
Numeric columns used: ['year_founded_target', 'employees_target', 'funding_target', 'acquirer_experience']


/Users/isaiah/aetos/lib/python3.12/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/isaiah/aetos/lib/python3.12/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/isaiah/aetos/lib/python3.12/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [24]:
display(merged_phase1)

,deal_value,year_founded_target,sector_target,employees_target,funding_target,ipo_status,country_target,State / Region (HQ)_Acquired,Acquiring Company,acquirer_experience,country_acquirer,Year of acquisition announcement,Deal announced on,Status,Terms,is_censored,lower_bound
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[24]7,NaN,NaN,2012,1/02/2012,Undisclosed,Undisclosed,True,625000000.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3Com,NaN,NaN,1997,1/06/1997,Undisclosed,Undisclosed,True,625000000.0
2,NaN,NaN,NaN,NaN,NaN,NaN,Canada,Ontario,Adobe,NaN,NaN,2002,15/04/2002,Undisclosed,Undisclosed,True,625000000.0
15,NaN,1984.0,Software,NaN,NaN,NaN,United States,Washington,Adobe,NaN,NaN,1994,31/08/1994,Undisclosed,Undisclosed,True,625000000.0
28,NaN,NaN,Software,NaN,NaN,NaN,United States,California,Adobe,NaN,NaN,2006,28/06/1905,Undisclosed,Undisclosed,True,625000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20108,20000000.0,2004.0,"Curated Web, Photo Sharing, Search, Email, Soc...",NaN,NaN,NaN,United States,California,Yahoo,NaN,NaN,2009,23/07/2009,Undisclosed,Cash,False,625000000.0
20123,29600000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yahoo,NaN,NaN,1998,1/12/1998,Undisclosed,Cash,False,625000000.0
20138,350000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yahoo,NaN,NaN,2007,1/09/2007,Undisclosed,Cash,False,625000000.0
20153,NaN,2012.0,"Hospitality, Restaurants, Travel, Mobile",NaN,NaN,NaN,United States,California,Yahoo,NaN,NaN,2014,12/08/2014,Undisclosed,Undisclosed,True,625000000.0


## Solver for Censored Regression

In [ ]:
# !pip install cvxpy
import numpy as np
import cvxpy as cp

# --- Sanitize arrays ---
# Convert to numpy just in case
X_unc = np.asarray(X_uncensored, dtype=float)
X_cen = np.asarray(X_censored, dtype=float)
y_unc = np.asarray(y_uncensored, dtype=float)

# Replace NaN/Inf and cap extreme magnitudes
def clean(A, cap=1e3):
    A = np.nan_to_num(A, nan=0.0, posinf=cap, neginf=-cap)
    A[np.abs(A) > cap] = np.sign(A[np.abs(A) > cap]) * cap
    return A

X_unc = clean(X_unc, cap=1e2)
X_cen = clean(X_cen, cap=1e2)
y_unc = clean(y_unc, cap=1e3)

# drop zero-variance columns so we can condition better
col_var = X_unc.var(axis=0)
keep = col_var > 1e-12
X_unc = X_unc[:, keep]
X_cen = X_cen[:, keep]
print(f"Kept {keep.sum()} / {keep.size} feature columns (dropped low-variance).")

# standardise y and threshold
y_mean, y_std = y_unc.mean(), (y_unc.std() if y_unc.std() > 1e-12 else 1.0)
y_sc = (y_unc - y_mean) / y_std
D_scaled = float(np.clip((D_val - y_mean) / y_std, -5.0, 5.0))

# define convex opti problem
n = X_unc.shape[1]
c = cp.Variable(n)

ridge = 1e-1  # regularisation strength
objective = cp.Minimize(cp.sum_squares(X_unc @ c - y_sc) + ridge * cp.sum_squares(c))

ridge = 1e0  # increase ridge term
result = prob.solve(
    solver=cp.OSQP,
    eps_abs=1e-4, eps_rel=1e-4,
    max_iter=30000,
    polish=True,
    verbose=True
)


constraints = []
if X_cen.shape[0] > 0:
    constraints = [X_cen @ c >= D_scaled]

prob = cp.Problem(objective, constraints)

# try OSQP first, fallback to SCS-indirect
try:
    res = prob.solve(
        solver=cp.OSQP,
        verbose=True,
        eps_abs=1e-3,
        eps_rel=1e-3,
        max_iter=20000,
        polish=True
    )
except (cp.SolverError, Exception):
    res = prob.solve(
        solver=cp.SCS,
        verbose=True,
        use_indirect=True,
        eps=1e-3,
        max_iters=5000,
        scale=0.1,
        acceleration_lookback=10
    )

print("Solver status:", prob.status)
print("Objective value:", res)

# rebuild full design and compute predictions
if prob.status in ("optimal", "optimal_inaccurate"):
    X_full = np.asarray(X, dtype=float)[:, keep]
    X_full = clean(X_full, cap=1e2)
    c_opt = c.value
    y_pred_sc = X_full @ c_opt
    y_pred = y_pred_sc * y_std + y_mean
    merged_phase1["predicted_price"] = y_pred
    print(merged_phase1["predicted_price"].describe())
else:
    print("⚠️ Solver did not converge. Status:", prob.status)

# check if constraints met
if X_cen.shape[0] > 0 and prob.status in ("optimal", "optimal_inaccurate"):
    viol = (X_cen @ c.value) < D_scaled - 1e-6
    print("Censored constraints violated on", int(viol.sum()), "rows (should be 0).")


(CVXPY) Oct 06 04:18:31 PM: Your problem has 201 variables, 1218 constraints, and 0 parameters.
(CVXPY) Oct 06 04:18:31 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Oct 06 04:18:31 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Oct 06 04:18:31 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Oct 06 04:18:31 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Oct 06 04:18:31 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Oct 06 04:18:31 PM: Finished problem compilation (took 2.829e-03 seconds).
(CVXPY) Oct 06 04:18:31 PM: Invoking solver OSQP  to obtain a solution.
(CVXPY) Oct 06 04:18:31 PM: Problem status: optimal
(CVXPY) Oct 06 04:18:31 PM: Optimal value: 8.724e+03
(CVXPY) Oct 06 04:18:31 PM: Compilation took 2.829e-03 seconds
(CVXPY) Oct 06 04:18:31 PM: Solver (includi

Kept 201 / 488 feature columns (dropped low-variance).
                                     CVXPY                                     
                                     v1.7.3                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
iter   objective    prim res   dual res   gap        rel kkt    rho         time
   1   8.7241e+03   1.48e-04   3.77e-04  -5.43e-02   3.77e-04   5.20e-01    7.30e-05s
  25   8.7242e+03   3.63e-05   1.64e-04  -1.16e-02   1.64e-04   5.20e-01    6.51e-04s

status:               solved
solution polishing:   

## Phase 2 - Covariance Estimation
### From Phase 1 we have:
- merged_phase1['predicted_price']  (proxy for monthly profit/value)
- 'Deal announced on' (date), and a company identifier
- Choose which entity drives the portfolio: 'Acquiring Company' or 'Acquired Company'.

In [44]:
df = merged_phase1.copy()

# parse time and choose the entity dimension
df['deal_date'] = pd.to_datetime(df['Deal announced on'], errors='coerce')
df = df.dropna(subset=['deal_date'])

# monthly bucket
df['month'] = df['deal_date'].dt.to_period('M').dt.to_timestamp()

# choose which side to model in portfolios
entity_col = 'Acquiring Company'   # or 'Acquired Company'

# build a monthly panel: sum across deals per (month, entity)
panel = (
    df.groupby(['month', entity_col], dropna=True)['predicted_price']
      .sum()                               # monthly total value/profit proxy
      .unstack(entity_col)                 # index=month, columns=entity
      .sort_index()
)

# keep entities with enough history
min_months = 6
panel = panel.loc[:, panel.notna().sum() >= min_months]

# fill months with no deals as zeros (interpreting “no M&A cashflow” as 0)
panel = panel.fillna(0.0)

# standardize per-column to control scale before cov
# for raw “profit” covariance, we can skip standardization, here i decided to demeans
# get deviation from monthly average profit hence subtract tryna get what the covariance measures
panel_centered = panel - panel.mean(axis=0)

# dompute sample mean and covariance (Markowitz inputs)
# μ: per-entity average monthly value; Σ: covariance across entities
mu = panel.mean(axis=0).to_numpy()             # shape (n_entities,)
Sigma_emp = np.cov(panel_centered.to_numpy(), rowvar=False, bias=False)  # (n x n)
entities = panel.columns.tolist()

print("Entities (n):", len(entities))
print("Panel shape (T x n):", panel.shape)
print("mu shape:", mu.shape, "| Sigma_emp shape:", Sigma_emp.shape)

# ==========================================================
# empirical covariances can be noisy or nearly-singular
# we can use precision matrix estimation to stabilize
# Sparse inverse covariance (Graphical-Lasso style)
# Solve:    min_{S ≻ 0}  -logdet(S) + tr(S * Σ_emp) + λ * ||S||_1_offdiag
# Returns a precision matrix S; can invert to get a stabilized covariance.
# ==========================================================

use_sparse_precision = True
lambda_glasso = 1e-2  # tune (start with 1e-2 to 1e-1; larger = sparser)

Sigma_glasso = None
S_val = None

if use_sparse_precision:
    n = Sigma_emp.shape[0]
    S = cp.Variable((n, n), symmetric=True)

    # Off-diagonal L1 penalty (don’t penalize the diagonal)
    offdiag = cp.norm1(S - cp.multiply(np.eye(n), S))

    # Objective: convex log-det program
    obj = cp.Minimize(-cp.log_det(S) + cp.trace(S @ Sigma_emp) + lambda_glasso * offdiag)

    # Constraint: S ≻ 0 (enforced via S >> 0 with a small PSD floor for numerics)
    constraints = [S >> 1e-6 * np.eye(n)]
    prob = cp.Problem(obj, constraints)
    _ = prob.solve(solver=cp.SCS, verbose=False, use_indirect=True, eps=1e-4, max_iters=5000)

    if prob.status in ("optimal", "optimal_inaccurate"):
        S_val = S.value
        # Invert S to get a stabilized covariance estimate
        # Add a tiny ridge for safety before inversion
        Sigma_glasso = np.linalg.pinv(S_val + 1e-8 * np.eye(n))
        print("Sparse precision estimated. Status:", prob.status)
        print("\n")
    else:
        print("Glasso solve status:", prob.status, "— using empirical Sigma only.\n")

print("Optimal inaccurate explains the resiudals hit the preset tolerance")
print("There is still a valid S (precision matrix) are invertible, PSD and usable for Phase 3")

Entities (n): 27
Panel shape (T x n): (207, 27)
mu shape: (27,) | Sigma_emp shape: (27, 27)
Sparse precision estimated. Status: optimal_inaccurate


Optimal inaccurate explains the resiudals hit the preset tolerance
There is still a valid S (precision matrix) are invertible, PSD and usable for Phase 3


/Users/isaiah/aetos/lib/python3.12/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


## Phase 3 - Portfolio Optimisation
### Inputs expected from Phase 2:
   - mu            : (n,) vector of expected monthly profits/returns
   - Sigma_emp     : (n,n) empirical covariance
   - Sigma_glasso  : (n,n) optional stabilised covariance (or None)
   - entities      : list of length n with asset/firm names

In [ ]:
# choose covariance & make it numerically safe
# make sure the covariance matrix is symmetric positive-semidefinite.
# If any eigenvalue is slightly negative (numerical noise), add a small ridge so the QP is strictly convex.
Sigma = Sigma_glasso if ('Sigma_glasso' in globals() and Sigma_glasso is not None) else Sigma_emp
Sigma = 0.5 * (Sigma + Sigma.T)                    # symmetrise
eigvals = np.linalg.eigvalsh(Sigma)
min_eig = eigvals.min()
if min_eig < 1e-8:
    Sigma += (1e-8 - min_eig) * np.eye(Sigma.shape[0])  # PSD floor

n = len(mu)
mu = np.asarray(mu).reshape(-1)                     # ensure shape (n,)
Sigma = np.asarray(Sigma, dtype=float)

# mean–variance scalarised QP
#   min_w  -mu^T w + gamma * w^T Sigma w
#   s.t.   1^T w = 1,  w >= 0

# First term (–μᵀw) = maximise expected return.
# Second term (γ wᵀΣw) = penalise risk (variance).
# γ controls risk-aversion:
# • small γ → chase returns,
# • large γ → prioritise safety.
# Constraints enforce a fully-invested, long-only portfolio.
def solve_markowitz_scalarized(mu, Sigma, gamma, solver="OSQP"):
    w = cp.Variable(n)
    obj = -mu @ w + gamma * cp.quad_form(w, Sigma)
    cons = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(cp.Minimize(obj), cons)
    try:
        _ = prob.solve(solver=getattr(cp, solver), verbose=False, max_iter=20000, polish=True)
    except Exception:
        _ = prob.solve(solver=cp.SCS, verbose=False, use_indirect=True, eps=1e-4, max_iters=8000, scale=0.1)
    return w.value, prob.status

# efficient frontier by sweeping gamma
# This sweeps 25 values of γ from 0.001 (return-hungry) to 100 (risk-averse), 
# building the efficient frontier – the trade-off curve of risk vs return.

gammas = np.logspace(-3, 2, 25)   # from return-seeking to risk-averse
frontier = []
weights_by_gamma = {}

for g in gammas:
    w_opt, status = solve_markowitz_scalarized(mu, Sigma, gamma=g, solver="OSQP")
    if status not in ("optimal", "optimal_inaccurate") or w_opt is None:
        continue
    w_opt = np.clip(w_opt, 0, None)  # guard tiny negatives
    w_opt = w_opt / w_opt.sum()      # renormalise
    ret = float(mu @ w_opt)
    risk = float(w_opt @ Sigma @ w_opt)             # portfolio variance
    frontier.append((g, ret, np.sqrt(risk)))
    weights_by_gamma[g] = w_opt

frontier_df = pd.DataFrame(frontier, columns=["gamma", "expected_return", "risk_std"]) # store each solutions y var, expected return and risk
print("Frontier points:", frontier_df.shape[0])
print(f"{frontier_df.shape[0]} feasible portoflios were found.")
display(frontier_df.head())

# pick a representative portfolio
# Example choices:
#   - most return-seeking (smallest gamma)
#   - mid gamma
#   - most risk-averse (largest gamma)
chosen_gamma = frontier_df.loc[frontier_df['gamma'].idxmin(), 'gamma']  # most return-seeking which is the smallest y
w_star = weights_by_gamma[chosen_gamma] # most aggressive investor
w_star = np.clip(w_star, 0, None); w_star = w_star / w_star.sum()

# summary stats for chosen_gamma
ret_star = float(mu @ w_star)
risk_star = float(w_star @ Sigma @ w_star)          # variance
summary = {
    "gamma": chosen_gamma,
    "expected_return": ret_star,
    "risk_std": np.sqrt(risk_star)
}
print("Chosen portfolio (most return-seeking):", summary)

# tidy weights table
w_tbl = pd.DataFrame({
    "entity": entities,
    "weight": w_star
}).sort_values("weight", ascending=False).reset_index(drop=True)

display(w_tbl.head(10))
print("Sum of weights:", w_tbl['weight'].sum())

# target-return min-variance variant
#   min_w  w^T Σ w
#   s.t.   μ^T w ≥ r_min, 1^T w = 1, w ≥ 0
# fix a required minimum return rₘᵢₙ (e.g. median μ) and find the portfolio with minimum risk that still achieves that return

def solve_markowitz_target(mu, Sigma, r_min, solver="OSQP"):
    w = cp.Variable(n)
    obj = cp.quad_form(w, Sigma)
    cons = [cp.sum(w) == 1, w >= 0, mu @ w >= r_min]
    prob = cp.Problem(cp.Minimize(obj), cons)
    try:
        _ = prob.solve(solver=getattr(cp, solver), verbose=False, max_iter=20000, polish=True)
    except Exception:
        _ = prob.solve(solver=cp.SCS, verbose=False, use_indirect=True, eps=1e-4, max_iters=8000, scale=0.1)
    return w.value, prob.status

# Example: set r_min to the median of μ
r_min = float(np.median(mu))
w_trg, status_trg = solve_markowitz_target(mu, Sigma, r_min, solver="OSQP")
if status_trg in ("optimal", "optimal_inaccurate") and w_trg is not None:
    w_trg = np.clip(w_trg, 0, None); w_trg = w_trg / w_trg.sum()
    print("Target-return portfolio solved. Return ≥", r_min)
    display(pd.DataFrame({"entity": entities, "weight": w_trg}).sort_values("weight", ascending=False).head(10))
else:
    print("Target-return problem status:", status_trg)


Frontier points: 25


,gamma,expected_return,risk_std
0,0.001000,388.208534,23.999501
1,0.001616,388.208534,23.999501
2,0.002610,388.208534,23.999501
3,0.004217,388.208534,23.999501
4,0.006813,388.208534,23.999501


Chosen portfolio (most return-seeking): {'gamma': np.float64(0.001), 'expected_return': 388.2085339980848, 'risk_std': np.float64(23.999500908467187)}


,entity,weight
0,Google,1.0
1,AOL,0.0
2,IBM,0.0
3,Yahoo,0.0
4,Twitter,0.0
5,Symantec,0.0
6,Salesforce,0.0
7,SAP,0.0
8,Qualcomm,0.0
9,Oracle,0.0


Sum of weights: 1.0
Target-return portfolio solved. Return ≥ 87.36942302837012


,entity,weight
9,Dropbox,0.218579
16,LinkedIn,0.072817
8,Dell,0.065050
20,Qualcomm,0.061116
22,Salesforce,0.049456
1,AT&T,0.046294
15,Intel,0.042048
19,Oracle,0.036707
23,Symantec,0.036209
2,Adobe,0.035705


In [47]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ensure results directory exists
os.makedirs("results", exist_ok=True)

# ==========================================================
# Export Efficient Frontier (risk vs expected return)
# ==========================================================
frontier_path = "results/frontier.csv"
frontier_df.to_csv(frontier_path, index=False)
print(f"Frontier saved to: {frontier_path}")

# ==========================================================
# Export Portfolio Weights
# ==========================================================
weights_path = "results/portfolio_weights.csv"
w_tbl.to_csv(weights_path, index=False)
print(f"Portfolio weights saved to: {weights_path}")

# ==========================================================
# Covariance Heatmap Plot
# ==========================================================
# Choose covariance matrix (stabilized if available)
Sigma_plot = Sigma_glasso if 'Sigma_glasso' in globals() and Sigma_glasso is not None else Sigma_emp

plt.figure(figsize=(10, 8))
plt.imshow(Sigma_plot, cmap="coolwarm", aspect="auto")
plt.title("Covariance Matrix Heatmap (Σ)", fontsize=14, pad=10)
plt.colorbar(label="Covariance value")
plt.xticks(ticks=np.arange(len(entities)), labels=entities, rotation=90)
plt.yticks(ticks=np.arange(len(entities)), labels=entities)
plt.tight_layout()

heatmap_path = "results/covariance_heatmap.png"
plt.savefig(heatmap_path, dpi=300, bbox_inches="tight")
plt.close()
print(f"Covariance heatmap saved to: {heatmap_path}")

# ==========================================================
# Final confirmation
# ==========================================================
print("All result files exported successfully.")
print(f"Frontier → {frontier_path}\nWeights → {weights_path}\nHeatmap → {heatmap_path}")


Frontier saved to: results/frontier.csv
Portfolio weights saved to: results/portfolio_weights.csv
Covariance heatmap saved to: results/covariance_heatmap.png
All result files exported successfully.
Frontier → results/frontier.csv
Weights → results/portfolio_weights.csv
Heatmap → results/covariance_heatmap.png


The optimiser placed 100 % weight on Google.  

Reason: with \( \gamma \) tiny, the risk penalty \( \gamma w^\top \Sigma w \) is negligible,  
so it simply picks the asset with the largest \( \mu \) (highest predicted average return).  
This is the corner portfolio on the extreme right of the frontier.  

Mathematically:  

$$
w^* = e_i \quad \text{for } i = \arg\max_i \mu_i
$$

For the **target return portfolio**, we have:

Here the constraint 

$$
\mu^\top w \ge 87.37
$$

forces diversification.  
The optimiser now minimises variance while satisfying that return floor.  
You get a spread across ~10 firms, each < 22 %, achieving the target with the lowest possible risk.  

**Economically:**  
Instead of betting everything on one high-return firm, the model balances exposure among correlated but complementary companies (cloud, hardware, telecom) to meet the same profit level more safely.



## Summary

| **Phase** | **Mathematical goal** | **What the output confirms** |
|:--|:--|:--|
| **1. Censored Regression** | Recover missing M\&A deal prices | Convex QP solved; full predicted dataset available |
| **2. Covariance Estimation** | Compute $$\mu$$ and $$\Sigma$$ of predicted profits | 27 entities $$\times$$ 207 months → well-conditioned covariance |
| **3. Portfolio Optimisation** | Use $$(\mu, \Sigma)$$ to build efficient portfolios | Frontier + optimal weights show how expected value vs risk trade-off |

**Convex pipeline summary:**

$$
\text{Incomplete data} 
\;\xrightarrow[\text{Phase 1}]{}\; 
\text{Estimated values} 
\;\xrightarrow[\text{Phase 2}]{}\; 
(\mu, \Sigma) 
\;\xrightarrow[\text{Phase 3}]{}\; 
\text{Optimal allocation } w
$$
